In [ ]:
import pandas as pd
from pathlib import Path
import sys
import plotly.express as px
import plotly.graph_objects as go

# Add the root project directory to the Python path
project_root = Path.cwd().parent  # This will get the project root since the notebook is in 'notebooks/'
sys.path.append(str(project_root))
from configs.path_config import OUTPUT_DIR, EXTRACTED_DATA_DIR
from src.clustering import clustering_preprocess

In [ ]:
path = OUTPUT_DIR / 'strain_distributions' / 'DPGMM' / 'C_step_size_7' / 'C_model.csv'
data_with_dpgmm = pd.read_csv(path, delimiter=',')
data_with_dpgmm.insert(
    loc=3,
    column='rolling_mean_prob',
    value=data_with_dpgmm['Assigned_Cluster_Prob'].rolling(window=7, min_periods=1).mean()
)

path = EXTRACTED_DATA_DIR / 'strain_distributions' / 'alvbrodel_04' / 'S-C_Close_Comp_20091129120000_20210611160000_strain_distribution_04.csv'
# path = OUTPUT_DIR / 'strain_distributions' / 'N-F_Mid_Comp_20091129120000_20210611160000_strain_distribution.csv'
df = clustering_preprocess.load_data(path)

In [ ]:
df = clustering_preprocess.drop_columns_by_header_rules(df, threshold=0)
df_strain, df = clustering_preprocess.remove_outliers(df, threshold=7, individual_threshold=7)

n_components = 8
normalized_pca_components, df_pca = clustering_preprocess.do_pca(n_components, df_strain, df)

In [ ]:
fig = px.line(data_with_dpgmm, x='Timestamp', y='Assigned_Cluster_Prob', color='Cluster', title='DPGMM Clustering Probabilities')
fig.update_traces(marker=dict(size=5))